# Functional Demostration - with TCPi

In [1]:
from pathfinder import Pathfinder

# abs_paths = [' ']
# Pathfinder.relative_paths_from_abs(abs_paths)
 
relative_paths = [['..', '..', '..', '..', '..', '..', '已完成', 'Bridges', 'bitbucket', 'github', 'codes'],
                  ['..', '..', '..', '..', '..', 'SigmaDSP', 'bitbucket', 'github', 'codes'],
                  ['..', '..', '..', '..', '..', 'Signal Generators', 'bitbucket', 'github', 'codes'],
                  ['..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],
                  ['..', '..', 'codes']] 
Pathfinder.append_relative_paths(relative_paths)

In [2]:
import threading
import time

from bridges.ftdi.controllers.i2c import I2cController
from sigma.bus import adapters
from tcpi.bus.tcpi_client import I2C as TcpI2C_client
from tcpi.bus.tcpi_server import I2C as TcpI2C_server
from tcpi.protocols.TCPIP1701 import class_finder

In [3]:
import pandas as pd

#https://thispointer.com/python-pandas-how-to-display-full-dataframe-i-e-print-all-rows-columns-without-truncation/
pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)

## TCPi server test

In [4]:
with_hardware_device = True

if with_hardware_device:
    ctrl = I2cController()
    _i2c = ctrl.I2C()

else:
    _i2c = None  # using None for testing without actual hardware device.

bus = adapters.I2C(i2c = _i2c)

### Server

In [5]:
tcpi_server = TcpI2C_server(bus, class_finder, i2c_addresses = {1: 0x68 >> 1, 2: 0xA0 >> 1})
print(tcpi_server.ip_address)
t_server = threading.Thread(target = tcpi_server.run)
t_server.start()
time.sleep(1)

('192.168.0.168', 8086)

[Server waiting for connection.]


### Client

In [6]:
tcpi_client = TcpI2C_client(class_finder)
tcpi_client.connect(*tcpi_server.ip_address)
time.sleep(1)


[Connecting server: ('192.168.0.168', 8086)]

[Connected with server: ('192.168.0.168', 8086)]
[Connect from client ('192.168.0.168', 61319) established.]



In [7]:
import os

from sigma.sigma_dsp.adau import ADAU1401

## Debug mode?

In [8]:
cls = ADAU1401

cls.DEBUG_MODE_SHOW_BUS_DATA = False         # whether to show bus data. 
cls.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## DSP processor

In [9]:
dsp = ADAU1401(bus = tcpi_client)


[Server receiving: 12 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 38, 0, 0]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2086
[ Data ]           :  [0, 0]
[ Data in Hex ]    :  ['0x0', '0x0']

[Server receiving: 156 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 34, 0, 0, 9, 1, 0, 0, 12, 52, 0, 2, 8, 34, 0, 0, 9, 1, 0, 0, 12, 52, 0, 2, 8, 34, 0, 0, 9, 1, 0, 0, 12, 52, 0, 2, 8, 34, 0, 0, 9, 1, 0, 0, 12, 52, 0, 2, 8, 34, 0, 0, 9, 1, 0, 0, 12, 52, 0, 2, 8, 34, 0, 0, 9, 1, 0, 0, 12, 52, 0, 2, 8, 34, 0, 0, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 16, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 24, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 28, 9, 1, 0, 0, 12, 52, 0, 2, 8, 39, 0, 1, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 28, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 28]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_len

## SigmaStudio project file

In [10]:
SigmaStudio_project_file_url = os.sep.join(['..', '..', 'SigmaStudio projects', 'projects', 'demo', 'demo.dspproj'])
SigmaStudio_project_file_url

'..\\..\\SigmaStudio projects\\projects\\demo\\demo.dspproj'

![](https://github.com/Wei1234c/SigmaDSP/blob/master/SigmaStudio%20projects/projects/demo/demo.png?raw=true)

[![SigmaDSP PC Control Test](https://raw.githubusercontent.com/Wei1234c/SigmaDSP/master/jpgs/demo_equip_setup.png)](https://youtu.be/XHlZtDsa4oE)     

## Factory

In [11]:
import os
from sigma.factory import Factory 

project_xml_file_url = os.sep.join(['..', '..', '..', '..', '..', 'SigmaDSP', 'bitbucket', 'github', 'SigmaStudio projects', 'projects', 'demo', 'demo.xml'])
class_files_root_url = os.sep.join( ['..', '..', '..', '..', '..', 'SigmaDSP', 'bitbucket', 'github', 'codes', 'sigma', 'sigma_studio', 'toolbox', 'cells']) 

factory = Factory(project_xml_file_url = project_xml_file_url,
                  class_files_root_url = class_files_root_url,
                  dsp = dsp
                 )

## IC

In [12]:
# showing cells, algorithms, parameters of this project

ic = factory.get_ic()
ic.df

param_full_name  \
algorithm_name       cell_name            param_name                                
FIRFiltAlg           FIR0                 fircoeff_0        FIRFiltAlg1fircoeff_0   
                     FIR1                 fircoeff_0        FIRFiltAlg2fircoeff_0   
                     FIR2                 fircoeff_0        FIRFiltAlg3fircoeff_0   
                     FIR3                 fircoeff_0        FIRFiltAlg4fircoeff_0   
Gain1940AlgNS        Gain_in0                                      Gain1940AlgNS3   
                     Gain_in1                                      Gain1940AlgNS2   
                     Gain_in2                                      Gain1940AlgNS7   
                     Gain_in3                                      Gain1940AlgNS8   
                     Gain_out0                                     Gain1940AlgNS1   
                     Gain_out1                                     Gain1940AlgNS4   
                     Gain_out2                                     Gain1940AlgNS5   
                     Gain_out3                                     Gain1940AlgNS6   
MonoNoPostGain3dBFix Compressor0                            MonoNoPostGain3dBFix1   
                                          RMS            MonoNoPostGain3dBFix1RMS   
                                          decay        MonoNoPostGain3dBFix1decay   
                                          hold          MonoNoPostGain3dBFix1hold   
                                          level        MonoNoPostGain3dBFix1level   
                     Compressor1                            MonoNoPostGain3dBFix2   
                                          RMS            MonoNoPostGain3dBFix2RMS   
                                          decay        MonoNoPostGain3dBFix2decay   
                                          hold          MonoNoPostGain3dBFix2hold   
                                          level        MonoNoPostGain3dBFix2level   
                     Compressor2                            MonoNoPostGain3dBFix3   
                                          RMS            MonoNoPostGain3dBFix3RMS   
                                          decay        MonoNoPostGain3dBFix3decay   
                                          hold          MonoNoPostGain3dBFix3hold   
                                          level        MonoNoPostGain3dBFix3level   
                     Compressor3                            MonoNoPostGain3dBFix4   
                                          RMS            MonoNoPostGain3dBFix4RMS   
                                          decay        MonoNoPostGain3dBFix4decay   
                                          hold          MonoNoPostGain3dBFix4hold   
                                          level        MonoNoPostGain3dBFix4level   
MultCtrlDelGrowAlg   Delay0                                   MultCtrlDelGrowAlg1   
                     Delay1                                   MultCtrlDelGrowAlg2   
                     Delay2                                   MultCtrlDelGrowAlg3   
                     Delay3                                   MultCtrlDelGrowAlg4   
MuteNoSlewAlg        Mute000              mute                 MuteNoSlewAlg1mute   
                     Mute001              mute                 MuteNoSlewAlg4mute   
                     Mute002              mute                 MuteNoSlewAlg7mute   
                     Mute003              mute                MuteNoSlewAlg10mute   
                     Mute010              mute                 MuteNoSlewAlg2mute   
                     Mute011              mute                 MuteNoSlewAlg6mute   
                     Mute012              mute                 MuteNoSlewAlg9mute   
                     Mute013              mute                MuteNoSlewAlg12mute   
                     Mute100              mute                MuteNoSlewAlg17mute   
                     Mute101              mute                MuteNoSle

## Cells 

In [13]:
cells = factory.get_cells()


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 8]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  8

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  8
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 0]
[ Data in Hex ]  :  ['0x0', '0x0', '0x0', '0x0']

[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 9]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  9

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  9
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 2]
[ Data in Hex ]  :  ['0x0', '0x0', '0x0', '0x2']


### Get Cell objects ready for use.

In [14]:
## un-comment this to generate a script to embody Cell objects.

# for o in factory.get_cells_manifest():
#     print(o) 

In [15]:
# assign variable names to each Cell for easy access.

compressor0 = cells['Compressor0']  # Mono RMS Full Range (No Post Gain)( 1 )
compressor1 = cells['Compressor1']  # Mono RMS Full Range (No Post Gain)( 1 )
compressor2 = cells['Compressor2']  # Mono RMS Full Range (No Post Gain)( 1 )
compressor3 = cells['Compressor3']  # Mono RMS Full Range (No Post Gain)( 1 )
delay0 = cells['Delay0']  # Del_Grow( 1 )
delay1 = cells['Delay1']  # Del_Grow( 1 )
delay2 = cells['Delay2']  # Del_Grow( 1 )
delay3 = cells['Delay3']  # Del_Grow( 1 )
fir0 = cells['FIR0']  # FIR( 1 )
fir1 = cells['FIR1']  # FIR( 1 )
fir2 = cells['FIR2']  # FIR( 1 )
fir3 = cells['FIR3']  # FIR( 1 )
gain_in0 = cells['Gain_in0']  # Gain (no slew)( 1 )
gain_in1 = cells['Gain_in1']  # Gain (no slew)( 1 )
gain_in2 = cells['Gain_in2']  # Gain (no slew)( 1 )
gain_in3 = cells['Gain_in3']  # Gain (no slew)( 1 )
gain_out0 = cells['Gain_out0']  # Gain (no slew)( 1 )
gain_out1 = cells['Gain_out1']  # Gain (no slew)( 1 )
gain_out2 = cells['Gain_out2']  # Gain (no slew)( 1 )
gain_out3 = cells['Gain_out3']  # Gain (no slew)( 1 )
merger0 = cells['Merger0']  # Signal MixerC
merger1 = cells['Merger1']  # Signal MixerC
merger2 = cells['Merger2']  # Signal MixerC
merger3 = cells['Merger3']  # Signal MixerC
mute000 = cells['Mute000']  # No Slew (Standard)( 1 )
mute001 = cells['Mute001']  # No Slew (Standard)( 1 )
mute002 = cells['Mute002']  # No Slew (Standard)( 1 )
mute003 = cells['Mute003']  # No Slew (Standard)( 1 )
mute010 = cells['Mute010']  # No Slew (Standard)( 1 )
mute011 = cells['Mute011']  # No Slew (Standard)( 1 )
mute012 = cells['Mute012']  # No Slew (Standard)( 1 )
mute013 = cells['Mute013']  # No Slew (Standard)( 1 )
mute100 = cells['Mute100']  # No Slew (Standard)( 1 )
mute101 = cells['Mute101']  # No Slew (Standard)( 1 )
mute102 = cells['Mute102']  # No Slew (Standard)( 1 )
mute103 = cells['Mute103']  # No Slew (Standard)( 1 )
mute110 = cells['Mute110']  # No Slew (Standard)( 1 )
mute111 = cells['Mute111']  # No Slew (Standard)( 1 )
mute112 = cells['Mute112']  # No Slew (Standard)( 1 )
mute113 = cells['Mute113']  # No Slew (Standard)( 1 )
mute_in0 = cells['Mute_in0']  # No Slew (Standard)( 1 )
mute_in1 = cells['Mute_in1']  # No Slew (Standard)( 1 )
mute_in2 = cells['Mute_in2']  # No Slew (Standard)( 1 )
mute_in3 = cells['Mute_in3']  # No Slew (Standard)( 1 )
mute_out0 = cells['Mute_out0']  # No Slew (Standard)( 1 )
mute_out1 = cells['Mute_out1']  # No Slew (Standard)( 1 )
mute_out2 = cells['Mute_out2']  # No Slew (Standard)( 1 )
mute_out3 = cells['Mute_out3']  # No Slew (Standard)( 1 )
param_eq_in0 = cells['Param EQ_in0']  # PEQ1Chan - Double Precision( 1 )
param_eq_in1 = cells['Param EQ_in1']  # PEQ1Chan - Double Precision( 1 )
param_eq_in2 = cells['Param EQ_in2']  # PEQ1Chan - Double Precision( 1 )
param_eq_in3 = cells['Param EQ_in3']  # PEQ1Chan - Double Precision( 1 )
param_eq_out0 = cells['Param EQ_out0']  # PEQ1Chan - Double Precision( 1 )
param_eq_out1 = cells['Param EQ_out1']  # PEQ1Chan - Double Precision( 1 )
param_eq_out2 = cells['Param EQ_out2']  # PEQ1Chan - Double Precision( 1 )
param_eq_out3 = cells['Param EQ_out3']  # PEQ1Chan - Double Precision( 1 )
source_switch_0 = cells['Source_Switch_0']  # Stereo SW Slew( 3 )
source_switch_1 = cells['Source_Switch_1']  # Stereo SW Slew( 3 )
tone1 = cells['Tone1']  # Tone Synthesis (lookup/sine)( 1 )
tone2 = cells['Tone2']  # Tone Synthesis (lookup/sine)( 1 )
volume_control_in0 = cells['Volume_Control_in0']  # Gain (RC Slew)( 1 )
volume_control_in1 = cells['Volume_Control_in1']  # Gain (RC Slew)( 1 )
volume_control_out01 = cells['Volume_Control_out01']  # Gain (RC Slew)( 1 )
volume_control_out23 = cells['Volume_Control_out23']  # Gain (RC Slew)( 1 )
white_noise = cells['White_Noise']  # White Noise( 1 )

## Functional Demostration

In [16]:
# switch to sine tone

source_switch_0.switch(0)


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 8]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  8

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  8
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 0]
[ Data in Hex ]  :  ['0x0', '0x0', '0x0', '0x0']

[Server receiving: 27 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 22, 0, 8, 9, 1, 0, 0, 15, 52, 0, 5, 8, 17, 0, 0, 0, 0, 0]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2070
[ Data ]           :  [0, 8]
[ Data in Hex ]    :  ['0x0', '0x8']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  15
[ Chip_address ]   :  52
[ Data_length ]    :  5
[ Addr

In [17]:
# get cell by name

tone2 = factory.get_cell_by_name('Tone2')

In [18]:
# disable sine tones

tone1.enable(False)
tone2.enable(False)


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 2]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  2

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  2
[ Success ]      :  0
[ Data ]         :  [0, 128, 0, 0]
[ Data in Hex ]  :  ['0x0', '0x80', '0x0', '0x0']

[Server receiving: 47 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 23, 0, 2, 9, 1, 0, 0, 15, 52, 0, 5, 8, 18, 0, 0, 0, 0, 0, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 60, 10, 0, 8, 52, 0, 4, 0, 5]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2071
[ Data ]           :  [0, 2]
[ Data in Hex ]    :  ['0x0', '0x2']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_len

In [19]:
# enable sine tones

tone1.enable(True)
tone2.enable(True)


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 2]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  2

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  2
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 0]
[ Data in Hex ]  :  ['0x0', '0x0', '0x0', '0x0']

[Server receiving: 12 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 25, 0, 2]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2073
[ Data ]           :  [0, 2]
[ Data in Hex ]    :  ['0x0', '0x2']

[Server receiving: 35 bytes of data.]
[9, 1, 0, 0, 15, 52, 0, 5, 8, 20, 0, 0, 128, 0, 0, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 60, 10, 0, 8, 52, 0, 4, 0, 5]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :

In [20]:
# set frequency 

tone1.set_frequency(440)
tone2.set_frequency(436)


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 0]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  0

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  0
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 255]
[ Data in Hex ]  :  ['0x0', '0x0', '0x0', '0xff']

[Server receiving: 12 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 22, 0, 0]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2070
[ Data ]           :  [0, 0]
[ Data in Hex ]    :  ['0x0', '0x0']

[Server receiving: 23 bytes of data.]
[9, 1, 0, 0, 15, 52, 0, 5, 8, 17, 0, 0, 0, 0, 255, 10, 0, 8, 52, 0, 4, 0, 1]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_l

In [21]:
# set frequency 

tone1.set_frequency(1100)
tone2.set_frequency(1104)


[Server receiving: 27 bytes of data.]
[9, 1, 0, 0, 15, 52, 0, 5, 8, 20, 0, 0, 2, 83, 73, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 60]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  15
[ Chip_address ]   :  52
[ Data_length ]    :  5
[ Address ]        :  2068
[ Data ]           :  [0, 0, 2, 83, 73]
[ Data in Hex ]    :  ['0x0', '0x0', '0x2', '0x53', '0x49']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2076
[ Data ]           :  [0, 60]
[ Data in Hex ]    :  ['0x0', '0x3c']

[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 0]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  0

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4


In [22]:
# switch to white noise

source_switch_0.switch(1)


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 8]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  8

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  8
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 0]
[ Data in Hex ]  :  ['0x0', '0x0', '0x0', '0x0']

[Server receiving: 12 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 25, 0, 8]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2073
[ Data ]           :  [0, 8]
[ Data in Hex ]    :  ['0x0', '0x8']

[Server receiving: 27 bytes of data.]
[9, 1, 0, 0, 15, 52, 0, 5, 8, 20, 0, 0, 0, 0, 1, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 60]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0


In [23]:
# decrease volume

volume_control_in0.set_dB(-20)


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 10]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  10

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  10
[ Success ]      :  0
[ Data ]         :  [0, 128, 0, 0]
[ Data in Hex ]  :  ['0x0', '0x80', '0x0', '0x0']

[Server receiving: 12 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 21, 0, 10]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2069
[ Data ]           :  [0, 10]
[ Data in Hex ]    :  ['0x0', '0xa']

[Server receiving: 27 bytes of data.]
[9, 1, 0, 0, 15, 52, 0, 5, 8, 16, 0, 0, 12, 204, 204, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 60]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_n

In [24]:
# increase volume

volume_control_in0.set_dB(0)


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 10]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  10

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  10
[ Success ]      :  0
[ Data ]         :  [0, 12, 204, 204]
[ Data in Hex ]  :  ['0x0', '0xc', '0xcc', '0xcc']

[Server receiving: 12 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 22, 0, 10]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2070
[ Data ]           :  [0, 10]
[ Data in Hex ]    :  ['0x0', '0xa']

[Server receiving: 27 bytes of data.]
[9, 1, 0, 0, 15, 52, 0, 5, 8, 17, 0, 0, 128, 0, 0, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 60]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_

In [25]:
# mute input

mute_in0.mute(True)
mute_in1.mute(True)


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 14]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  14

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  14
[ Success ]      :  0
[ Data ]         :  [0, 128, 0, 0]
[ Data in Hex ]  :  ['0x0', '0x80', '0x0', '0x0']

[Server receiving: 27 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 23, 0, 14, 9, 1, 0, 0, 15, 52, 0, 5, 8, 18, 0, 0, 0, 0, 0]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2071
[ Data ]           :  [0, 14]
[ Data in Hex ]    :  ['0x0', '0xe']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  15
[ Chip_address ]   :  52
[ Data_length ]    :  

In [26]:
# un-mute input

mute_in0.mute(False)
mute_in1.mute(False)


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 14]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  14

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  14
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 0]
[ Data in Hex ]  :  ['0x0', '0x0', '0x0', '0x0']

[Server receiving: 12 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 25, 0, 14]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2073
[ Data ]           :  [0, 14]
[ Data in Hex ]    :  ['0x0', '0xe']

[Server receiving: 35 bytes of data.]
[9, 1, 0, 0, 15, 52, 0, 5, 8, 20, 0, 0, 128, 0, 0, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 60, 10, 0, 8, 52, 0, 4, 0, 15]

<< Write >>          
[ Control ]        :  9
[ Safeload ]  

In [27]:
# disable white noise

white_noise.enable(False)


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 6]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  6

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  6
[ Success ]      :  0
[ Data ]         :  [0, 128, 0, 0]
[ Data in Hex ]  :  ['0x0', '0x80', '0x0', '0x0']

[Server receiving: 12 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 22, 0, 6]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2070
[ Data ]           :  [0, 6]
[ Data in Hex ]    :  ['0x0', '0x6']

[Server receiving: 27 bytes of data.]
[9, 1, 0, 0, 15, 52, 0, 5, 8, 17, 0, 0, 0, 0, 0, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 60]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] : 

In [28]:
# enable white noise

white_noise.enable(True)


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 6]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  6

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  6
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 0]
[ Data in Hex ]  :  ['0x0', '0x0', '0x0', '0x0']

[Server receiving: 12 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 23, 0, 6]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2071
[ Data ]           :  [0, 6]
[ Data in Hex ]    :  ['0x0', '0x6']

[Server receiving: 27 bytes of data.]
[9, 1, 0, 0, 15, 52, 0, 5, 8, 18, 0, 0, 128, 0, 0, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 60]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  

In [29]:
# set input0 dB = 0

gain_in0.set_dB(0)


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 38]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  38

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  38
[ Success ]      :  0
[ Data ]         :  [0, 128, 0, 0]
[ Data in Hex ]  :  ['0x0', '0x80', '0x0', '0x0']

[Server receiving: 12 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 24, 0, 38]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2072
[ Data ]           :  [0, 38]
[ Data in Hex ]    :  ['0x0', '0x26']

[Server receiving: 27 bytes of data.]
[9, 1, 0, 0, 15, 52, 0, 5, 8, 19, 0, 0, 128, 0, 0, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 60]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_num

In [30]:
# set input0 dB = -10

gain_in0.set_dB(-10)


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 38]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  38

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  38
[ Success ]      :  0
[ Data ]         :  [0, 128, 0, 0]
[ Data in Hex ]  :  ['0x0', '0x80', '0x0', '0x0']

[Server receiving: 39 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 25, 0, 38, 9, 1, 0, 0, 15, 52, 0, 5, 8, 20, 0, 0, 40, 122, 38, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 60]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2073
[ Data ]           :  [0, 38]
[ Data in Hex ]    :  ['0x0', '0x26']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  15
[ 

In [31]:
# read input0 dB value

gain_in0.get_dB()


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 38]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  38

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  38
[ Success ]      :  0
[ Data ]         :  [0, 40, 122, 38]
[ Data in Hex ]  :  ['0x0', '0x28', '0x7a', '0x26']


-10.000002514143702

In [32]:
# read input0 gain value

gain_in0.get_gain()


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 38]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  38

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  38
[ Success ]      :  0
[ Data ]         :  [0, 40, 122, 38]
[ Data in Hex ]  :  ['0x0', '0x28', '0x7a', '0x26']


0.31622767448425293

In [33]:
# set input0 gain = 1

gain_in0.set_gain(1)


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 38]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  38

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  38
[ Success ]      :  0
[ Data ]         :  [0, 40, 122, 38]
[ Data in Hex ]  :  ['0x0', '0x28', '0x7a', '0x26']

[Server receiving: 12 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 21, 0, 38]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2069
[ Data ]           :  [0, 38]
[ Data in Hex ]    :  ['0x0', '0x26']

[Server receiving: 27 bytes of data.]
[9, 1, 0, 0, 15, 52, 0, 5, 8, 16, 0, 0, 128, 0, 0, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 60]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel

In [34]:
# check input0 dB value

gain_in0.get_dB()


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 38]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  38

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  38
[ Success ]      :  0
[ Data ]         :  [0, 128, 0, 0]
[ Data in Hex ]  :  ['0x0', '0x80', '0x0', '0x0']


0.0

In [35]:
# check input0 gain value

gain_in0.get_gain()


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 38]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  38

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  38
[ Success ]      :  0
[ Data ]         :  [0, 128, 0, 0]
[ Data in Hex ]  :  ['0x0', '0x80', '0x0', '0x0']


1.0

In [36]:
# read FIR0's coefficients

coeffs = fir0.get_coefficients()
coeffs


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 40, 0, 82]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  40
[ Address ]      :  82

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  49
[ Chip_address ] :  52
[ Data_length ]  :  40
[ Address ]      :  82
[ Success ]      :  0
[ Data ]         :  [0, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[ Data in Hex ]  :  ['0x0', '0x80', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']


[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [37]:
# set and check FIR0's coefficients

fir0.set_coefficients([1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0])
fir0.get_coefficients()


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 40, 0, 82]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  40
[ Address ]      :  82

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  49
[ Chip_address ] :  52
[ Data_length ]  :  40
[ Address ]      :  82
[ Success ]      :  0
[ Data ]         :  [0, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[ Data in Hex ]  :  ['0x0', '0x80', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']

[Server receiving: 58 bytes of data.]
[9, 1, 0, 0, 50, 52, 0, 40, 0, 82, 0, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

[1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0]

In [38]:
# coefficients of FIR0 can also be accessed via. get_table() method.

fir0.get_table().numbers


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 40, 0, 82]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  40
[ Address ]      :  82

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  49
[ Chip_address ] :  52
[ Data_length ]  :  40
[ Address ]      :  82
[ Success ]      :  0
[ Data ]         :  [0, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[ Data in Hex ]  :  ['0x0', '0x80', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x1', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']


[1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0]

In [39]:
# set delay samples

delay0.set_delayed_samples(1)


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 122]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  122

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  122
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 1]
[ Data in Hex ]  :  ['0x0', '0x0', '0x0', '0x1']

[Server receiving: 12 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 22, 0, 122]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2070
[ Data ]           :  [0, 122]
[ Data in Hex ]    :  ['0x0', '0x7a']

[Server receiving: 27 bytes of data.]
[9, 1, 0, 0, 15, 52, 0, 5, 8, 17, 0, 0, 0, 0, 1, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 60]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_num

In [40]:
# set delay samples

delay1.set_delayed_samples(1)


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 123]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  123

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  123
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 1]
[ Data in Hex ]  :  ['0x0', '0x0', '0x0', '0x1']

[Server receiving: 12 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 23, 0, 123]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2071
[ Data ]           :  [0, 123]
[ Data in Hex ]    :  ['0x0', '0x7b']

[Server receiving: 27 bytes of data.]
[9, 1, 0, 0, 15, 52, 0, 5, 8, 18, 0, 0, 0, 0, 1, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 60]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_num

In [41]:
# read and check delay samples

delay0.get_delayed_samples()


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 122]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  122

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  122
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 1]
[ Data in Hex ]  :  ['0x0', '0x0', '0x0', '0x1']


1

In [42]:
# read and check delay milliseconds

delayed_ms = delay0.get_delayed_ms()
delayed_ms


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 122]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  122

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  122
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 1]
[ Data in Hex ]  :  ['0x0', '0x0', '0x0', '0x1']


0.020833333333333332

In [43]:
# set delay milliseconds

delay0.set_delayed_ms(delayed_ms * 2)


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 122]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  122

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  122
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 1]
[ Data in Hex ]  :  ['0x0', '0x0', '0x0', '0x1']

[Server receiving: 12 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 24, 0, 122]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2072
[ Data ]           :  [0, 122]
[ Data in Hex ]    :  ['0x0', '0x7a']

[Server receiving: 27 bytes of data.]
[9, 1, 0, 0, 15, 52, 0, 5, 8, 19, 0, 0, 0, 0, 2, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 60]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_num

In [44]:
# read and check delay samples

delay0.get_delayed_samples()


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 122]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  122

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  122
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 2]
[ Data in Hex ]  :  ['0x0', '0x0', '0x0', '0x2']


2

In [45]:
# read PEQ0's coefficients

coeffs = param_eq_in0.get_parameters_values()
coeffs


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 18]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  18

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  18
[ Success ]      :  0
[ Data ]         :  [0, 128, 55, 92]
[ Data in Hex ]  :  ['0x0', '0x80', '0x37', '0x5c']

[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 19]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  19

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  19
[ Success ]      :  0
[ Data ]         :  [15, 0, 144, 116]
[ Data in Hex ]  :  ['0xf', '0x0', '0x90', '0x74']

[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 20]

<< ReadRequest >>  
[ Control ]      :  1

[['0B1', 1.0016894340515137],
 ['1A1', 1.995591640472412],
 ['1B1', -1.995591640472412],
 ['2A1', -0.9956583976745605],
 ['2B1', 0.9939688444137573]]

In [46]:
# set and check PEQ0's coefficients

coeffs[2][1] = -1.0

param_eq_in0.set_parameters_values(coeffs)
param_eq_in0.get_parameters_values()


[Server receiving: 8 bytes of data.]
[10, 0, 8, 52, 0, 4, 0, 18]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  18

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  13
[ Chip_address ] :  52
[ Data_length ]  :  4
[ Address ]      :  18
[ Success ]      :  0
[ Data ]         :  [0, 128, 55, 92]
[ Data in Hex ]  :  ['0x0', '0x80', '0x37', '0x5c']

[Server receiving: 12 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 25, 0, 18]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2073
[ Data ]           :  [0, 18]
[ Data in Hex ]    :  ['0x0', '0x12']

[Server receiving: 35 bytes of data.]
[9, 1, 0, 0, 15, 52, 0, 5, 8, 20, 0, 0, 128, 55, 92, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 60, 10, 0, 8, 52, 0, 4, 0, 21]

<< Write >>          
[ Control ]        :  9
[ Sa

[['0B1', 1.0016894340515137],
 ['1A1', 1.995591640472412],
 ['1B1', -1.0],
 ['2A1', -0.9956583976745605],
 ['2B1', 0.9939688444137573]]

In [47]:
# reload from EEPROM, equivalent to soft-reset

dsp.reload_from_eeprom()


[Server receiving: 8 bytes of data.]
[10, 0, 8, 80, 1, 244, 0, 0]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  0

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  0
[ Success ]      :  0
[ Data ]         :  [1, 0, 5, 0, 8, 28, 0, 88, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 0, 35, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 235, 0, 0, 0, 0, 0, 0, 255, 0, 0, 150, 48, 0, 128, 0, 0, 0, 0, 0, 255, 0, 0, 155, 166, 0, 128, 0, 0, 0, 128, 0, 0, 0, 5, 70, 75, 0, 0, 0, 0, 0, 0, 0, 2, 0, 128, 0, 0, 0, 0, 8, 0, 0, 128, 0, 0, 0, 0, 8, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 55, 92, 15, 0, 144, 116, 0, 127, 58, 95, 0, 255, 111, 140, 15, 128, 142, 68, 0, 128, 55, 92, 15, 0, 144, 116, 0, 127, 58, 95, 0, 255, 111, 140, 15, 


[Server receiving: 8 bytes of data.]
[10, 0, 8, 80, 1, 244, 3, 232]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  1000

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  1000
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 90, 130, 122, 0, 64, 0, 0, 0, 45, 65, 61, 0, 32, 0, 0, 0, 22, 160, 158, 0, 16, 0, 0, 0, 0, 19, 7, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 81, 236, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  2000
[ Success ]      :  0
[ Data ]         :  [226, 1, 255, 233, 8, 32, 1, 1, 240, 0, 226, 1, 1, 145, 8, 32, 1, 1, 153, 8, 34, 73, 255, 241, 8, 32, 38, 1, 240, 0, 226, 1, 1, 177, 8, 32, 1, 255, 241, 19, 34, 73, 1, 145, 8, 32, 1, 1, 209, 8, 32, 37, 1, 216, 0, 226, 1, 1, 193, 8, 32, 1, 255, 241, 19, 34, 73, 1, 145, 8, 32, 1, 1, 225, 8, 32, 37, 1, 232, 0, 226, 1, 1, 249, 8, 32, 1, 0, 0, 0, 0, 1, 2, 8, 0, 240, 1, 2, 9, 8, 32, 1, 255, 241, 19, 34, 73, 1, 241, 8, 32, 1, 255, 233, 8, 32, 37, 1, 240, 0, 226, 1, 1, 145, 8, 32, 1, 1, 152, 0, 226, 37, 1, 161, 8, 32, 1, 1, 177, 8, 34, 65, 2, 0, 0, 226, 1, 1, 177, 8, 32, 1, 2, 1, 20, 34, 1, 1, 184, 0, 226, 1, 1, 216, 0, 192, 1, 0, 7, 255, 160, 1, 1, 184, 0, 192, 1, 1, 103, 255, 32, 129, 1, 80, 0, 226, 1, 1, 127, 255, 64, 129, 1, 88, 0, 228, 1, 255, 241, 8, 32, 1, 255, 247, 255, 34, 65, 1, 200, 0, 226, 1, 1, 232, 0,


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  3000
[ Success ]      :  0
[ Data ]         :  [34, 1, 4, 98, 65, 52, 1, 4, 90, 66, 34, 1, 4, 82, 62, 34, 1, 4, 74, 63, 34, 1, 4, 66, 64, 34, 1, 0, 0, 0, 0, 1, 4, 104, 0, 226, 1, 4, 128, 0, 242, 1, 4, 194, 70, 32, 1, 4, 186, 71, 34, 1, 4, 170, 70, 52, 1, 4, 162, 71, 34, 1, 4, 154, 67, 34, 1, 4, 146, 68, 34, 1, 4, 138, 69, 34, 1, 0, 0, 0, 0, 1, 4, 176, 0, 226, 1, 4, 200, 0, 242, 1, 5, 10, 75, 32, 1, 5, 2, 76, 34, 1, 4, 242, 75, 52, 1, 4, 234, 76, 34, 1, 4, 226, 72, 34, 1, 4, 218, 73, 34, 1, 4, 210, 74, 34, 1, 0, 0, 0, 0, 1, 4, 248, 0, 226, 1, 5, 16, 0, 242, 1, 5, 82, 80, 32, 1, 5, 74, 81, 34, 1, 5, 58, 80, 52, 1, 5, 50, 81, 34, 1, 5, 42, 77, 34, 1, 5, 34, 78, 34, 1, 5, 26, 79, 34, 1, 0, 0, 0, 0, 1, 5, 64, 0, 226, 1, 5, 88, 0, 242, 1, 4, 105, 8, 32, 1, 5, 168, 0, 226, 1, 5, 98, 91, 32, 1, 5, 106, 90, 34, 1, 5, 114, 89, 34, 1, 5, 122, 88, 34, 1, 5, 130, 87


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  4000
[ Success ]      :  0
[ Data ]         :  [242, 1, 8, 40, 0, 192, 1, 0, 0, 0, 0, 1, 0, 2, 135, 161, 1, 255, 225, 8, 32, 1, 0, 2, 134, 161, 1, 255, 225, 8, 34, 65, 7, 248, 0, 226, 1, 8, 48, 0, 192, 1, 7, 255, 255, 32, 1, 255, 225, 8, 34, 1, 8, 8, 0, 226, 1, 8, 8, 0, 192, 1, 7, 71, 255, 32, 1, 7, 176, 0, 226, 1, 255, 242, 221, 64, 1, 7, 72, 0, 192, 1, 7, 79, 255, 32, 1, 0, 0, 0, 0, 1, 8, 184, 0, 226, 1, 8, 192, 0, 242, 1, 7, 73, 12, 32, 1, 8, 224, 0, 226, 1, 8, 224, 0, 192, 1, 8, 231, 255, 32, 1, 8, 224, 0, 226, 1, 8, 233, 8, 32, 1, 8, 234, 218, 34, 65, 8, 226, 218, 34, 9, 8, 240, 0, 226, 1, 8, 113, 8, 32, 1, 8, 114, 218, 34, 65, 8, 194, 218, 34, 1, 8, 97, 8, 52, 1, 8, 98, 218, 34, 65, 8, 186, 218, 34, 1, 8, 104, 0, 226, 1, 255, 233, 8, 66, 1, 8, 120, 0, 242, 1, 8, 241, 8, 32, 1, 255, 233, 8, 130, 1, 255, 233, 8, 132, 35, 255, 241, 10, 32, 1, 0, 0, 0

...


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  12000
[ Success ]      :  0
[ Data ]         :  [58, 174, 34, 9, 8, 72, 0, 226, 1, 7, 201, 8, 32, 1, 7, 202, 174, 34, 65, 8, 26, 174, 34, 1, 7, 185, 8, 52, 1, 7, 186, 174, 34, 65, 8, 18, 174, 34, 1, 7, 192, 0, 226, 1, 255, 233, 8, 66, 1, 7, 208, 0, 242, 1, 8, 73, 8, 32, 1, 255, 233, 8, 130, 1, 255, 233, 8, 132, 35, 255, 241, 10, 32, 1, 0, 0, 0, 0, 1, 8, 80, 0, 246, 1, 8, 81, 8, 34, 1, 8, 32, 0, 226, 1, 0, 0, 0, 0, 1, 8, 32, 0, 246, 35, 7, 233, 8, 32, 9, 7, 240, 0, 226, 1, 7, 217, 8, 32, 1, 255, 242, 176, 34, 103, 7, 224, 0, 226, 1, 8, 33, 8, 34, 73, 8, 33, 8, 32, 1, 7, 224, 0, 226, 39, 255, 242, 175, 32, 1, 7, 240, 0, 226, 39, 0, 0, 0, 0, 1, 7, 241, 8, 32, 9, 255, 249, 8, 34, 65, 7, 240, 0, 226, 38, 7, 225, 26, 32, 1, 0, 0, 0, 0, 1, 8, 40, 0, 226, 1, 8, 48, 0, 242, 1, 8, 40, 0, 192, 1, 0, 0, 0, 0, 1, 0, 2, 135, 161, 1, 255, 225, 8, 32, 1, 0, 2, 134, 161


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  13000
[ Success ]      :  0
[ Data ]         :  [193, 8, 32, 1, 9, 195, 50, 34, 65, 10, 19, 50, 34, 1, 9, 177, 8, 52, 1, 9, 179, 50, 34, 65, 10, 11, 50, 34, 1, 9, 184, 0, 226, 1, 255, 233, 8, 66, 1, 9, 200, 0, 242, 1, 10, 65, 8, 32, 1, 255, 233, 8, 130, 1, 255, 233, 8, 132, 35, 255, 241, 10, 32, 1, 0, 0, 0, 0, 1, 10, 72, 0, 246, 1, 10, 73, 8, 34, 1, 10, 24, 0, 226, 1, 0, 0, 0, 0, 1, 10, 24, 0, 246, 35, 9, 225, 8, 32, 9, 9, 232, 0, 226, 1, 9, 209, 8, 32, 1, 255, 243, 52, 34, 103, 9, 216, 0, 226, 1, 10, 25, 8, 34, 73, 10, 25, 8, 32, 1, 9, 216, 0, 226, 39, 255, 243, 51, 32, 1, 9, 232, 0, 226, 39, 0, 0, 0, 0, 1, 9, 233, 8, 32, 9, 255, 249, 8, 34, 65, 9, 232, 0, 226, 38, 9, 217, 26, 32, 1, 0, 0, 0, 0, 1, 10, 32, 0, 226, 1, 10, 40, 0, 242, 1, 10, 32, 0, 192, 1, 0, 0, 0, 0, 1, 0, 3, 11, 161, 1, 255, 225, 8, 32, 1, 0, 3, 10, 161, 1, 255, 225, 8, 34, 65, 9, 240,


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  14000
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  15000
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  393
[ Chip_address ] :  80
[ Data_length ]  :  384
[ Address ]      :  16000
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### Close TCP channels

In [48]:
tcpi_client.stop() 

tcpi_server.stop()
t_server.join()


[Client set to stop.]

[Server set to stop ]

[Connection with client ('192.168.0.168', 61319) is closed.]
